In [0]:
# Run the all the agents notebook to initiate the my_agent_function. Only the one selected
# in cofig variable AGENT_FRAMEWORK will be run and the rest will be skipped

In [ ]:
%run "./agents/agent_langchain.ipynb"

In [ ]:
%pip install --upgrade numpy pandas scikit-learn

In [4]:
%pip uninstall mlflow mlflow-genai -y
%pip install mlflow==2.13.0
#%pip install mlflow[genai]  # or, alternatively: pip install mlflow==2.12.1 mlflow-genai



Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 133.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.2
    Uninstalling protobuf-5.28.2:
      Successfully uninstalled protobuf-5.28.2
Note: you may need to restart the kernel to use updated packages.


In [73]:
import mlflow
print("MLflow version:", mlflow.__version__)
import mlflow.models as models
print(dir(models))

MLflow version: 2.13.0
['EvaluationArtifact', 'EvaluationMetric', 'EvaluationResult', 'FlavorBackend', 'MetricThreshold', 'Model', 'ModelConfig', 'ModelInputExample', 'ModelSignature', 'Resource', 'ResourceType', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'add_libraries_to_model', 'build_docker', 'dependencies_schema', 'evaluate', 'evaluation', 'flavor_backend', 'flavor_backend_registry', 'get_model_info', 'infer_pip_requirements', 'infer_signature', 'list_evaluators', 'make_metric', 'model', 'model_config', 'predict', 'python_api', 'resources', 'set_model', 'set_retriever_schema', 'set_signature', 'signature', 'utils', 'validate_schema']


In [9]:
%pip install mlflow[genai]



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:


# IMPORTANT: After running the above line, restart the kernel to ensure all packages are reloaded properly.
# This will resolve the numpy dtype size changed error.  mlflow.models.llmd  doesnt exist in mlflow.models

from typing import Callable, Generator, List

import mlflow
from agent_config import *
# from mlflow.models import set_model  # Removed: set_model does not exist in mlflow.models
from mlflow.pyfunc import ChatModel
from mlflow.types.llm import (
    ChatCompletionResponse,
    ChatMessage,
    ChatParams,
)
 

In [75]:
from typing import Callable, Generator, List

import mlflow
from agent_config import *  # assuming you have config like model path, etc.

from mlflow.pyfunc import PythonModel  # Use base class
from mlflow.models import infer_signature  # for evaluation metrics/logging


## Setting up your agent for MLFlow registration

In [76]:
from typing import Callable, Generator, List, Optional, Dict, Any
import mlflow.pyfunc


class ChatMessage:
    def __init__(self, role: str, content: str):
        self.role = role
        self.content = content


class ChatParams:
    def __init__(self, temperature: float = 0.7, max_tokens: int = 1024, custom_inputs: Optional[Dict[str, Any]] = None):
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.custom_inputs = custom_inputs or {}


class ChatCompletionResponse:
    def __init__(self, response: str, metadata: Optional[Dict[str, Any]] = None):
        self.response = response
        self.metadata = metadata or {}


In [77]:
class MyAgent(mlflow.pyfunc.PythonModel):
    """
    Defines a custom agent that processes ChatCompletionRequests
    and returns ChatCompletionResponses.
    """

    def __init__(self, agent_name: str, agent_function: Callable):
        self.agent_name = agent_name
        self.agent_function = agent_function

    def _get_client_type(self, params: ChatParams) -> str:
        if params.custom_inputs and "client_type" in params.custom_inputs:
            return params.custom_inputs["client_type"]
        return "unknown"

    def predict(self, context, model_input: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Accepts a list of inputs where each item is a dict containing:
        - 'messages': List[Dict[str, str]] (with 'role' and 'content')
        - 'params': Optional[Dict] with keys like 'temperature', 'max_tokens', 'custom_inputs'

        Returns:
            A list of dicts with a 'response' field and optional 'metadata'.
        """
        results = []
        for item in model_input:
            messages = [ChatMessage(**m) for m in item["messages"]]
            param_data = item.get("params", {})
            params = ChatParams(**param_data)

            res = self.agent_function(messages=messages, params=params, verbose=True)
            results.append({
                "response": res.response,
                "metadata": res.metadata,
            })
        return results


In [78]:
def dummy_agent_function(messages: List[ChatMessage], params: ChatParams, verbose=False) -> ChatCompletionResponse:
    last_user_msg = next((m.content for m in reversed(messages) if m.role == "user"), "")
    reply = f"Echo: {last_user_msg} (temp={params.temperature})"
    return ChatCompletionResponse(response=reply, metadata={"echoed": True})


In [79]:
from mlflow.models.signature import infer_signature
import pandas as pd

# Use dummy flattened input for schema inference
example_input = pd.DataFrame([{
    "messages": '[{"role": "user", "content": "hello"}]',
    "params": '{"temperature": 0.5}'
}])
example_output = pd.DataFrame([{"response": "Echo: hello", "metadata": '{"echoed": true}'}])

signature = infer_signature(example_input, example_output)


In [81]:
import mlflow.pyfunc

agent = MyAgent(agent_name="echo-agent", agent_function=dummy_agent_function)

mlflow.pyfunc.save_model(
    path="my_agent_model_v2",
    python_model=agent,
    artifacts=None,
    signature=signature,
    conda_env=mlflow.pyfunc.get_default_conda_env()
)


In [85]:
import mlflow
#from my_agent_model import MyAgent

# Dummy function to simulate response generation
def dummy_agent_function(messages: List[ChatMessage], params: ChatParams, verbose=False) -> ChatCompletionResponse:
    return ChatCompletionResponse(response=f"Processed {len(messages)} messages", metadata={"dummy": True})


model = MyAgent(agent_name="TestAgent", agent_function=dummy_agent_function)

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="my_agent_model",
        python_model=model,
        registered_model_name="MyAgentModel",
        input_example=[
            {
                "messages": [{"role": "user", "content": "Hello"}],
                "params": {"temperature": 0.5, "max_tokens": 256, "custom_inputs": {}}
            }
        ],
    )



2025/05/22 21:41:52 INFO mlflow.types.utils: Unsupported type hint: typing.List[typing.Dict[str, typing.Any]], skipping schema inference
2025/05/22 21:41:52 INFO mlflow.types.utils: Unsupported type hint: typing.List[typing.Dict[str, typing.Any]], skipping schema inference
2025/05/22 21:41:52 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to infer schema for pandas.Series 0    {'temperature': 0.5, 'max_tokens': 256, 'custo...\nName: params, dtype: object. Error: Creating Object with empty properties is not allowed."). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`. To disable automatic signature inference, set `signature` to `False` in your `log_model` or `save_model` call.
2025/05/22 21:41:54 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
TypeError('strin

In [87]:
with mlflow.start_run() as run:
    run_id = run.info.run_id
    mlflow.pyfunc.log_model(
        artifact_path="my_agent_model",
        python_model=model,
        signature=signature
    )
    print(f"Run ID: {run_id}")


Run ID: cf094287-d05b-4265-b8c0-7241e61d2e39


In [ ]:
eval_data = pd.DataFrame([
    {
        "messages": [
            {"role": "user", "content": "How many white patients in the dataset?"},
            {"role": "assistant", "content": "300"},
            {"role": "user", "content": "How many hispanic patients in the dataset?"},
            {"role": "assistant", "content": "400"},
            {"role": "user", "content": "What race was the last one I asked about?"}
        ],
        "params": {"custom_inputs": {"client_type": "mobile"}},
        "target": "Hispanic"  # Expected answer
    }
])



In [96]:
import pandas as pd
from mlflow.models import evaluate


results = evaluate(
    model="runs:/cf094287-d05b-4265-b8c0-7241e61d2e39/my_agent_model",
    data=eval_data,
    targets="target",  # <== name of the column, not a list
    model_type="text",  # Probably more appropriate than "classifier" for LLMs
    evaluators="default",
)
print(results.metrics)


2025/05/22 21:55:57 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.


TypeError: string indices must be integers

In [ ]:
import pandas as pd
from mlflow.models import evaluate

# Test data
eval_data = pd.DataFrame([
    {"messages": [{"role": "user", "content": "Hi"}], "params": {}},
    {"messages": [{"role": "user", "content": "What’s the weather?"}], "params": {}}
])

results = evaluate(
    model="runs:/cf094287-d05b-4265-b8c0-7241e61d2e39/my_agent_model",  # or use the local path if not registered
    data=eval_data,
    targets=["Processed 1 messages", "Processed 1 messages"],  # Expected outputs
    model_type="classifier",  # You can also use 'regressor' or 'text'
    evaluators="default",
)

print(results.metrics)


MlflowException: The targets argument must be specified for classifier models.

After this different code

In [4]:
import pandas as pd
from typing import Any

class MyChatModel(mlflow.pyfunc.PythonModel):
    def predict(self, context: mlflow.pyfunc.PythonModelContext, model_input: pd.DataFrame) -> list[str]:
        # Your model logic here
        return ["Echo: " + str(msg) for msg in model_input["message"]]



In [5]:
from mlflow.models import infer_signature

example_input = pd.DataFrame({"message": ["Hello"]})
example_output = ["Echo: Hello"]
signature = infer_signature(example_input, example_output)

mlflow.pyfunc.log_model(
    artifact_path="chat_model",
    python_model=MyChatModel(),
    signature=signature,
    input_example=example_input,
)


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
2025/05/21 14:48:41 INFO mlflow.pyfunc: Validating input example against model signature


In [83]:
class MyAgent(ChatModel):
    """
    Defines a custom agent that processes ChatCompletionRequests
    and returns ChatCompletionResponses.
    """

    def __init__(
        self,
        agent_name: str,
        agent_function: Callable,
    ) -> None:
        self.agent_name = agent_name
        self.agent_function = agent_function

    def predict(
        self, context, messages: list[ChatMessage], params: ChatParams
    ) -> ChatCompletionResponse:
        # Get a "client_type" field from params.custom_inputs, if specified.
        # It's best practice to assume custom_inputs is optional and may not be provided
        client_type = self._get_client_type(params)
        res = self.agent_function(messages=messages, params=params, verbose=True)
        return res

    def _get_client_type(self, params: ChatParams) -> str:
        """
        Helper for extracting the "client type" used to make a request to the
        agent. The client type is passed as custom input to the agent
        """
        if (
            client_type := params
            and params.custom_inputs
            and params.custom_inputs.get("client_type", "unknown")
        ):
            return client_type
        return "unknown"

    def predict_stream(
        self, context, messages: List[ChatMessage], params: ChatParams
    ) -> Generator[ChatCompletionChunk, None, None]:
        raise NotImplementedError("Streaming agents are not supported by this model.")

NameError: name 'ChatCompletionChunk' is not defined

In [7]:
agent = MyAgent(agent_name=MODEL_NAME, agent_function=my_agent_function)
set_model(agent)

In [8]:
# Run your agent class to test the execution

input_messages = [
    ChatMessage(role="user", content="How many white patients in the dataset?"),
    ChatMessage(role="assistant", content="300"),
    ChatMessage(role="user", content="How many hispanic patients in the dataset?"),
    ChatMessage(role="assistant", content="400"),
    ChatMessage(role="user", content="What race was the last one I asked about?"),
]
params_with_custom_inputs = ChatParams(custom_inputs={"client_type": "mobile"})
response = agent.predict(context=None, messages=input_messages, params=params_with_custom_inputs)
print(response.choices[0].message.content)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


The race you asked about last was Hispanic.

> Finished chain.
The race you asked about last was Hispanic.


## Registering your model through MLFlow

In [ ]:
metadata = None

registered_model_name = MODEL_NAME

In [ ]:
# Then run an experiment to log the model in MLFLow


input_messages = [
    ChatMessage(role="user", content="Hi"),
]

with mlflow.start_run() as r:

    logged_agent_info = mlflow.pyfunc.log_model(
        python_model=agent,
        artifact_path=ARTIFACT_PATH,
        registered_model_name=registered_model_name,
        input_example=input_messages,
        metadata=metadata,
        pip_requirements="requirements.txt",
    )

2025/02/27 18:58:53 INFO mlflow.pyfunc: Predicting on input example to validate output
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Registered model 'agent_demo' already exists. Creating a new version of this model...
2025/02/27 18:59:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: agent_demo, version 18
Created version '18' of model 'agent_demo'.


Hello! How can I assist you today? If you have any questions or need information, feel free to ask.

> Finished chain.
Hello! How can I assist you today? If you have any questions or need information, feel free to ask.

> Finished chain.
🏃 View run goofy_island_2p1szkcy at: https://eastus2.api.azureml.ms/mlflow/v2.0/subscriptions/91c3532a-a293-4c32-a909-b3be52cb3084/resourceGroups/codeorange-edap-us6-shared-svc-appresources-devtest/providers/Microsoft.MachineLearningServices/workspaces/coedapssus6azmldevtest001/#/experiments/f1a1e24c-4df5-4026-ba6c-f3dcfd1bc9c8/runs/25cb70c7-3f8e-4ba2-8258-36df14001800
🧪 View experiment at: https://eastus2.api.azureml.ms/mlflow/v2.0/subscriptions/91c3532a-a293-4c32-a909-b3be52cb3084/resourceGroups/codeorange-edap-us6-shared-svc-appresources-devtest/providers/Microsoft.MachineLearningServices/workspaces/coedapssus6azmldevtest001/#/experiments/f1a1e24c-4df5-4026-ba6c-f3dcfd1bc9c8


In [ ]:
# WORK IN PROGRESS
# working on the code to create a serving endpoint for your Agent in AML